# pip install this
0 to PyPI in 60 minutes

In [1]:
# Pay no heed to the man behind the terminal

import os
start_dir = os.path.expanduser('~/work/pip-install-this')
os.chdir(start_dir)

! ./cleanup.sh
! ssh-add -D
! ssh-add ~/.ssh/pip-install-this-id_rsa

stuff_dir = os.path.join(start_dir, 'hippy-chat-stuff')
os.chdir(stuff_dir)

All identities removed.
Identity added: /Users/tbeck/.ssh/pip-install-this-id_rsa (/Users/tbeck/.ssh/pip-install-this-id_rsa)


## About Me
I'm Tucker Beck.
* twitter: [@tuckerbeck](https://twitter.com/tuckerbeck)
* github: [dusktreader](https://github.com/dusktreader)

I'm a senior backend software developer with some full-stack tendencies.

I work for [comScore](https://www.comscore.com/). We measure and analyze media consumption at a global scale.

I love python, and I'd love to tell you why...after this talk!

## Introduction
It's a great time to be a developer! There are so many great languages, lots of cool emerging tools, tons of documentation, and it's all so easy to get!

The open-source community is really flourishing in the Python universe. But, getting your software published can seem like a very daunting task to new developers (and even veterans that haven't published before).

Lot of tools are emerging that make publishing your software easier with a lot less 'fiddling'. Removing the barrier of tedium should encourage more diversity (in all senses of the word) in our code ecosystem and developer community



## About this talk
This jupyter-notebook (and supporting files) are hosted on [github.com](https://github.com/dusktreader/pip-install-this/)

There's a lot of material to cover here, and the material is discussion worthy.

However, to get through everything I have prepared here, I have to ask for questions to be held until the end.

That being said, I look forward to discussing details, personal experiences, and individual perspectives when we get to the end of the talk.

So, buckle up, and get ready to drink from the firehose!

## Assumptions
For this talk, I'm going to assume the following are available:
* Python3
* git

I've also installed (ahead of time)
* virtualenv
* travis command-line tool (ruby)

And I have accounts created for:
* github.com
* readthedocs.org
* travis-ci.org
* pypi.python.com

## Some Thoughts
Python is an opinionated language. I'm an opinionated developer. This is fine, as long as it is understood.

I prefer Python 3. This talk is in python 3. That's all I have to say about that

Holy wars are fought over dogma, not philosophy. So, start with [The Zen of Python]( https://www.python.org/dev/peps/pep-0020/) and then try to find community accepted best-practices from there. 

Sometimes, it's better to accept a convention you don't personally like for the sake of consistency and accessibility.



## Other Thoughts
There is a lot of philosophy and discussion about packaging. Python software is most commonly distributed in packages on [PyPI](https://pypi.python.org/pypi), but there are alternative packagers, repositories, and schools of thought. We're not going to go into this here, but it is interesting! Instead, this talk is intended to be a pragmatic guide to getting your code published.

I've focused on simple and easy tools and procedures. Some of the things I talk about will be personal best practices. I'll call these out with _italics_. Your mileage may vary, and I understand and hope you have your own set of best-practices or preferences. I'm happy to talk about them later.

## What we're going to cover
* Basic python package
* Simple but thorough setup.py
* Rapid test suite creation
* Easy code style checking
* Quick documentation creation
* Fundamental github repo creation
* Automatic documentation page generation 
* Painless continous-integration tools
* Automatic publishing


## Starting from not quite 0
At comScore, we use Hipchat for team communication. I wanted to see critical errors in my team's hipchat channel (I'm lead dev...I get to decide that =P)

There weren't any extensions for python logging that do this yet, so I rolled my own using the very cool [Hipchat API](https://www.hipchat.com/docs/apiv2)

Since it's a useful tool, and there is nothing proprietary in it, I think it would be a good thing to get out into the Python OSS community.

So, in this talk, we're going to publish this logging handler as an independent python package


## The basic python package

There are, again, a lot of opinions about best-practices in structuring python packages. What I'm presenting here is a combination of best-practices, easy and convenient methodologies, and my peferred packages.

The package structure for our package we're building today is as follows:


### hippy-chat package layout
```
hippy-chat/
|--> setup.py
|--> MANIEST.in
|--> LICENSE.rst
|--> CHANGELOG.rst
|--> README.rst
|--> .project_metadata.json
|--> .hipchat.json
|--> hippy_chat
     |--> __init__.py
     |--> handler.py
     |--> exec/
          |--> demo.py
```

In [3]:
root_dir = os.path.expanduser('~/work/hippy-chat')
! mkdir $root_dir

## Support files

### The setup.py file for the project
Some things to note:
* I use the setup method from setuptools instead of distutils
* But, _I like to install with pip even locally_
* So, none of this stuff is tested using ```$ python setup.py install```
* I've used a lot of variations of setup.py structure, and this is the layout I've arrived at.
* I promise you I'm not done trying new things!

In [4]:
! cp $stuff_dir/setup.py $root_dir/ 
! cat $root_dir/setup.py

import json
import setuptools


with open('.project_metadata.json') as meta_file:
    project_metadata = json.loads(meta_file.read())


setuptools.setup(
    name=project_metadata['name'],
    version=project_metadata['release'],
    author=project_metadata['author'],
    author_email=project_metadata['author_email'],
    description=project_metadata['description'],
    license=project_metadata['license'],
    install_requires=[
        'arrow',
        'click',
        'py-buzz',
        'requests',
    ],
    extras_require={
        'dev': [
            'flake8',
            'pytest',
            'pytest-capturelog',
            'sphinx',
        ],
    },
    include_package_data=True,
    packages=setuptools.find_packages(),
    entry_points={
        'console_scripts': [
            'hippy-chat-demo = hippy_chat.exec.demo:main',
        ],
    },
)


### Some notes about setup.py

* setup.py provides info to both install **and** build (for publication) your package
* Name of the root directory doesn't actually matter
* _Don't put metadata in the base \__init\__.py_
* Only recently started putting executables in exec/
* I'll talk about .project_metadata.json later. This is a _personal peference_
* I use ReStructuredText for my documents, because it's what sphinx uses. _I also like it better than markdown_

#### Why the project_metadata dict?
```
    version=project_metadata['release'],
```
There's [lots and lots of discussion](https://www.google.com/search?q=python+best+practices+project+metadata) about how to include meta-data (especially version data) in your project. There doesn't seem to be a universally accepted best practice, so I came up with my own.

Keeping project metadata in the root project folder this way has some advantages:
* json. Everything can parse json.
* dynamic. I can change metadata (like version number) in one place
* other stuff can easily find it. Like sphinx, for instance (we'll see this later)

#### Why extras_require?
```
    extras_require={
        'dev': [
            'flake8',
            'pytest',
            'pytest-capturelog',
        ],
    },
```
Using the extras_require directive lets you have differing requirements for different roles. Basic installs shouldn't install dependencies that are only needed for dev, testing, document generation, etc.

To install using a different role, you can do it like this:
```
$ pip install <package>[dev]
```




#### Why include_package_data?
```
    include_package_data=True,
```

You'll need to include this to also install stuff like .project_metadata.json or any other files not a part of your source tree. These will need to be specified in MANIFEST.in.

**Note**: If you're reading through examples and/or the distutils/setuptools specification, you might find (and be rather tempted by) ```data_files=...```. I have not found this to work consistently between distutils, setuptools, and pip.

#### Why find_packages?
Because it's smart and really easy. Otherwise, you have to list your main package, and all sub-packages therein:
```
    packages=['a', 'a.a', 'a.a.a', 'a.b', 'a.b.a']
```
If you add additional sub-packages, you will have to remember to add them here. Of course, you won't notice until you publish to PyPI and try to pip install your package next time...


#### Why entry_points?
I only started doing this recently.

Previously, I kept all my executables in ```<root>/bin/``` and added a ```scripts=glob.blog('bin/*')```. This works ok, but there are a few things that make it frustrating
* No (good) way to import the scripts for unit testing
* Don't forget to add your shebang!
* Make sure your shebang calls out the right python version
* Make sure that your executables have the executable flag set
* Probably shouldn't name them with an extension (as is usual in unix for executable files)
* Not cross-platform

#### Why entry_points? (continued)
Using entry points gives you a lot of magic that takes care of those things (for the most part). Pip will create the executable in a platform specific way at install time, rename it, and make sure that it references the right python interpreter. Further, since the executables are part of the package itself, you can easily unit test them. Finally, a single module can have multiple entry points that are all split out into their own executables!

It should also be noted that this pattern works really well with click.

### The CHANGELOG.rst file
Changelogs are an important way for users to track what has changed between releases. This way, they can peruse major differences and decide if upgrading is worth the risk to their software. There is some really great discussion about this topic provided at [keepachagnelog.com](http://keepachangelog.com/en/0.3.0/). After reading through, I became a believer.

In [5]:
! cp $stuff_dir/CHANGELOG.rst $root_dir/
! cat $root_dir/CHANGELOG.rst

************
 Change Log
************

All notable changes to this project will be documented in this file.

The format is based on [Keep a Changelog](http://keepachangelog.com/)
and this project adheres to [Semantic Versioning](http://semver.org/).

Unreleased
----------
- This CHANGELOG
- README providing a brief overview of the project
- First release of hippy-chat


### The MANIFEST.in file
As stated above, this is the most consistent way to make sure that extra files are installed with your project. The file structure is pretty much dead simple. Just list exactly the extra files that you want to include

In [6]:
! cp $stuff_dir/MANIFEST.in $root_dir/
! cat $root_dir/MANIFEST.in

include .project_metadata.json
include .hipchat.json


### The LICENSE.rst file
Always add a license. For python packages, I usually like to use the MIT license because it's very permissive. However, you should [do some reading to decide which license to use](https://choosealicense.com/)

I like to format my license the same as my README and CHANGELOG, just for neurotic consistency

In [7]:
! cp $stuff_dir/LICENSE.rst $root_dir/
! cat $root_dir/LICENSE.rst

MIT License

Copyright (c) `2017` `Tucker Beck`

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CON

### The README.rst file
You really should have a README immediately. Notice no italics? This isn't an opinion. It's pretty important. Your README doesn't have to be really verbose. It also probably shouldn't be the difinitive documentation for your project (you should build those with sphinx...but more on that later). However, the README should be a touchstone for people who want to use your package. At a minimum it should include
* A summary, no matter how terse, of what your project is for and what it does
* Basic requirements. **Especially** which versions of python it supports
* An explanation of how to get an environment set up for development and testing

In [8]:
! cp $stuff_dir/README.rst $root_dir/
! cat $root_dir/README.rst

************
 hippy-chat
************

----------------------------------
Hipchat logging handler for python
----------------------------------

This package provides a logging handler that can send messages to a hipchat
room in the form of notifications.

Requirements

 - Python 3

Installing
Install using pip::

$ pip install hippy-chat

Using
=====
Setup the handler with the needed credentials for the hipchat server's api that
you will be sending messages to:

.. code-block:: python

   from hippy_chat.handler import HipchatHandler

   handler = HipchatHandler(
       'https://some_hipchat_server', 'some-room', 'SOME_HIPCHAT_API_TOKEN',
   )
   logger.addHandler(handler)

Contributing

In order to get your environment set up for development, it is recommended to
use virtualenv and pip to install dev dependencies::

    $ virtualenv --python=python3 env
    $ source env/bin/activate
    $ pip install -e .[dev]


### The .project_metadata.json file
As I explained above, this is a personal best-practice. However you do it, though, you should try to keep your project metadata together in one and only one place. This is especially important for project versions. You don't want to confuse your user when they see one version on your documentation and another on the package they download from PyPI. Using a single metadata file like this can help with these sorts of problems

In [9]:
! cp $stuff_dir/.project_metadata.json $root_dir/
! cat .project_metadata.json

{
    "author": "Tucker Beck",
    "author_email": "tucker.beck@gmail.com",
    "copyright": "2017, Tucker Beck",
    "description": "Log messages to a hipchat room",
    "license": "MIT",
    "name": "hippy-chat",
    "release": "0.1.1",
    "version": "0.1"
}


### The .hipchat.json file
This config file contains credentials that will be loaded by our demo executable to talk to my hipchat room. This one is just specific to this project

In [10]:
! cp $stuff_dir/.hipchat.json $root_dir/
! cat $root_dir/.hipchat.json

{
    "url": "https://pip-install-this.hipchat.com",
    "room": "Pip-install-this",
    "token": "c4Hp59gNczgP4Uor3gt2XklphLY5JFc3s2EZ6n1u"
}


## Source Code
There are a few best practices with how source code should be laid out:

* All source should go in a directory in the project root
* This directory should be in lower **snake_case**, and should be some derivation of your package name
* You should try to pick something unique enough that it won't collide with other packages 
* This directory name will be the importable name of your package
  
There are, of course, a lot of others. But these are the basics
  

### The hippy_chat handler module
This is the source for our chat handler. A few things to note:
* [arrow](http://crsmithdev.com/arrow/) is a really nice datetime replacement. _I recommend using this or [pendulum](https://github.com/sdispater/pendulum) for **all** cases where you work with datetimes_
* [py-buzz](https://github.com/dusktreader/py-buzz) is one of my packages that provides some nice add-ons to python exceptions
* [requests](http://docs.python-requests.org/en/master/) is an awesome package for working with apis. It's dead simple to use.

Also worth noting: This is a logging handler for normal python logging. This is what our project started using a long time ago. Python logging is pretty cool, but _it is not as cool as it could or should be. These days, I try to use [logbook](https://github.com/getlogbook/logbook) wherever possible_. However, logbook is not a drop-in replacement, so our project will probably stick with vanilla logging for now...or maybe forever

Also: We will also need to add an empty init.py module. This tells Python that this directory contains a package and, potentially, subpackages. Python won't be able to find the 'handlers' module without this

In [11]:
source_dir = os.path.join(root_dir, 'hippy_chat')

! mkdir $source_dir
! touch $source_dir/__init__.py
! cp $stuff_dir/handler.py $source_dir
! cat $source_dir/handler.py

import arrow
import buzz
import json
import logging
import requests
import textwrap
import urllib


HIPCHAT_DEFAULT_TITLE = 'event occured in application'


class HipchatHandler(logging.Handler):
    """
    This module provides a logging handler that sends notifications to a
    hipchat room. The notifications will appear as expandable 'cards'
    with relevant data.

    .. todo:: Add sentry support for the card urls
    """

    def __init__(
            self, url, room, token, title=HIPCHAT_DEFAULT_TITLE,
            *args, **kwargs
    ):
        """
        Initializes the handler

        :param: url:   The base URL to the hipchat server
        :param: room:  The name of the room in which to make the notification
        :param: token: The auth token for the room
        :param: title: An optional title to include in the notification
        """

        self.url = '{url}/v2/room/{room}/notification'.format(
            url=url,
            room=urllib.parse.quote(room),
      

## Test Code
Writing unit tests is a great way to verify you package's behavior even as you are writing it. There are [lots of good reasons](http://www.onjava.com/pub/a/onjava/2003/04/02/javaxpckbk.html) to write unit tests, but the biggest one is that you **will** miss obvious bugs, and unit tests help you find them before you even notice they are there

There are a few different python testing frameworks out there.

The unittest package is a part of the python standard library, and it offers a lot of features out of the box. However, I don't recommend it for a few reasons:
* Test discovery is painful
* Using the special assert methods requires a lot of lookup time that could be spent writing tests
* Everything **has** to be done in test classes and heirarchies thereof
* No pluggable extensions

For a long time, [nose](http://nose.readthedocs.io/en/latest/) addressed a lot of those concerns. However, nose is no longer under active development, and, its replacement, [nose2](https://github.com/nose-devs/nose2) is still in 'beta' (pre 1.0 release) and is not progressing very rapidly. Additionally, _nose introduces some of its own quirks that can be annoying_.

### Why pytest?
I find that [pytest](http://doc.pytest.org/en/latest/) is a great testing framework that allows you to get testing very rapidly and yet has enough features to support basically anything you want to do. There are a lot of useful pytest plugins out there as well to fill in missing functionality.

The pytest framework _can_ use test class heirarchies, but they are optional. [Most setup in pytest is done via fixtures](http://doc.pytest.org/en/latest/fixture.html), which can be incredibly powerful (if somewhat confusing at times)

Project wide and directory wide configuration is available via `conftest.py` files, but we won't need any for hippy-chat, and you may not need them for your project

### Where to put tests
Tests can be:
* interleaved with your source code
* put in a subdirectory of your source directory (as a module)
* put in their own directory in the project root

The most common practice, and my preferred approach, is to put the tests in their own directory immediately below the root of your project. This makes them very easy to target with your command line and also lowers the risk of accidental imports.

### The test_handler.py test module
This module contains the unit tests for the hippy_chat.handler module. This example is pretty spartan, but there should be enough here to show some pytest features.


In [12]:
test_dir = os.path.join(root_dir, 'tests')

! mkdir $test_dir
! touch $test_dir/__init__.py
! cp $stuff_dir/test_handler.py $test_dir
! cat $test_dir/test_handler.py

import collections
import json
import logging
import pytest
import requests

from hippy_chat.handler import HipchatHandler, HIPCHAT_DEFAULT_TITLE


class TestHipchatHandler:

    @pytest.fixture(autouse=True)
    def setup(self):
        """
        Sets up the logger for the tests in this class. Is automatically
        invoked before each test function is called
        """
        self.logger = logging.getLogger('TestHipchatLogger')
        self.logger.setLevel(logging.DEBUG)

    @pytest.yield_fixture
    def dummy_handler(self):
        """
        Provides a dummy instance of a HipchatHandler
        """
        dummy_url = 'https://some.fake.server.hipchat.com'
        dummy_room_name = 'some-room'
        dummy_token = 'SOME_TOKEN'

        dummy_handler = HipchatHandler(dummy_url, dummy_room_name, dummy_token)
        self.logger.addHandler(dummy_handler)
        yield dummy_handler
        self.logger.removeHandler(dummy_handler)

    def test___init__(self, dummy_handler):
 

### More about pytest
Invoking pytest against our project is incredibly simple. You simply need to call it and target the test directory. You can target specific tests, test directories, or tests matching a specific pattern.

We'll invoke tests in a bit after we are done setting up the package

## Executables
Executable code _should be as compact as possible_. It's good practice to make your entry-points concise and keep most logic within your python modules. Ideally, an executable does little more than parse a command line, call into a function, and then possibly produce output

I use [click](https://github.com/pallets/click) for command line processing. _This is a matter of personal preference_. However, you should **never do your own command line processing**. Even for really simple stuff, I reccommend using a command line parser. [argparse](https://docs.python.org/3/library/argparse.html) (and its predecessor, [optparse](https://docs.python.org/2/library/optparse.html)) is built into the language and provides a full-featured command-line parser.

### The demo.py executable file
This module will just be used to show a demonstration of the logging handler in action. It parses some basic command line options and does some logging to show the handler working.

We'll run this demo a bit later.

In [13]:
exec_dir = os.path.join(source_dir, 'exec')

! mkdir $exec_dir
! touch $exec_dir/__init__.py
! cp demo.py $exec_dir
! cat demo.py

import arrow
import click
import json
import logging
import os
import time
import hippy_chat.handler


@click.command()
@click.option(
    '-c', '--config-file',
    default='.hipchat.json',
    help="the file containing hipchat deets",
)
@click.option(
    '-l', '--log-level',
    default='ERROR',
    type=click.Choice(['DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL']),
    help="select the level of messges that should be logged to the room",
)
@click.option(
    '-i', '--message-interval',
    default=5,
    type=int,
    help="How often to message the room",
)
def main(config_file, log_level, message_interval):

    log_level = getattr(logging, log_level)

    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)

    with open(os.path.expanduser('.hipchat.json')) as config_file:
        config = json.load(config_file)
    hipchat_handler = hippy_chat.handler.HipchatHandler(**config)
    hipchat_handler.setLevel(log_level)

    logger.addHandler(hipchat_handler)

  

## Working within a virtual environment
Python's virtual environments are just too nice to _not_ use. They are really simple to set up, and provide a good way to isolate your environment as you develop. Using virtual environments has the following benefits:
* Improve repeatability in package setup
* Easy _removal_ of your installed package
* Provide custom configuration to your package without breaking anything else
* Use whatever version of python interpreter you want

### Setting up the virtual environment
Note that I'm making the wild assumption that you have [virtualenv](https://virtualenv.pypa.io/en/stable/) set up. If not, the setup is pretty easy. That does, however, depend on your os

Creating and activating the virtualenv is pretty easy. Once it is done, invoking python (or a script that invokes python via a shebang line) in the active virtual environment will be working in its own encapsulated environment. This also means that any packages that are installed via pip will be localized to this virtual environment

In [14]:
os.chdir(root_dir)

! virtualenv --python=python3 env
! source env/bin/activate

Running virtualenv with interpreter /Users/tbeck/.virtualenvs/pip-install-this/bin/python3
Using real prefix '/usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5'
New python executable in /Users/tbeck/work/hippy-chat/env/bin/python3.5
Also creating executable in /Users/tbeck/work/hippy-chat/env/bin/python
Installing setuptools, pip, wheel...done.


## Installing the new package in 'edit' mode
While you are actively developing a project, it is very convenient to have it installed inside of your active virtual environment in 'editable' mode. Pip offers this feature so that you can modify a source code and see the changes reflected the next time you invoke python or a python script without having to re-install the package. Trust me, you want to use this feature when you are working on your package

Here we will invoke the install command against the pwd with the 'dev' modifier. This will install all of the normal package dependencies but will also install the dependencies from the 'dev' section of 'extras_require' in our setup.py:

Once the install is finished, our environment has all the dependencies satisfied including the hippy-chat package that we are building

In [15]:
! pip install -e .[dev]

Obtaining file:///Users/tbeck/work/hippy-chat
  Found existing installation: hippy-chat 0.1.1
    Uninstalling hippy-chat-0.1.1:
      Successfully uninstalled hippy-chat-0.1.1
  Running setup.py develop for hippy-chat


## Invoking the test suite

The pytest framework has a whole slew of options that you can specify on the command line. Two options I find most useful are the --maxfail and --verbose options

I generally invoke pytest against my entire test suite using the `--maxfail=1` option. This tells pytest to stop immediately upon the first test failure. This is useful for larger projects where you want to see a problem right away when a test fails. You usually won't set this if you are, for instance, running your test suite as a deployment step or any time you want a report of all failing tests

You can also increase the verbosity of failure messages by passing `--verbose` flags. The flags are additive, so two verbose flags will show even more verbose output.

Note: As of this writing, I discovered some weirdness with python interpreter and py.test vs pytest

In [16]:
! py.test --maxfail=1 --verbose $test_dir

============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-3.0.6, py-1.4.32, pluggy-0.4.0 -- /Users/tbeck/.virtualenvs/pip-install-this/bin/python3.5
cachedir: .cache
rootdir: /Users/tbeck/work/hippy-chat, inifile: 
plugins: capturelog-0.7
collected 3 items 

tests/test_handler.py::TestHipchatHandler::test___init__ PASSED
tests/test_handler.py::TestHipchatHandler::test__make_payload PASSED
tests/test_handler.py::TestHipchatHandler::test_integration PASSED

============================ pytest-warning summary ============================
WI1 /Users/tbeck/.virtualenvs/pip-install-this/lib/python3.5/site-packages/pytest_capturelog.py:171 'pytest_runtest_makereport' hook uses deprecated __multicall__ argument
WC1 None pytest_funcarg__caplog: declaring fixtures using "pytest_funcarg__" prefix is deprecated and scheduled to be removed in pytest 4.0.  Please remove the prefix and use the @pytest.fixture decorator instead.
WC1 None p

## Trying out the demo
Now that everything is setup, we should be able to try out the demo. Note, that it is installed into our editable virtualenv, and is invokable by its installed name: 'hippy-chat-demo'

We'll watch for the results in [a hipchat room I set up for this talk](https://pip-install-this.hipchat.com/chat/room/3517101)

In [ ]:
! hippy-chat-demo --log-level WARNING --message-interval=2

## Code Style

Style checking is great! It takes the thinking out of code formatting. I used to like to develop my own style and tinker with it. That's a lot of thought put into something that _doesn't really matter that much_. What I've found is that using an established style-guide let's me dedicate that mind space to working on my package. It also prevents 'discussions' about style within my team. Even if no one quite agrees, having some 3rd party style guide helps keep the team working on implementation.

Having a consitent code style that is automatically checked can also help new developers start contributing consistent code and will help open source contributors know how to style their contributions.

So, I like to use vanilla pep8 via out-of-the-box [flake8](https://pypi.python.org/pypi/flake8)

### Why flake8?
There are a few different style-checkers out there that also work well. Choosing one is a matter of perference. Most of them do the same linting, but they all have different sets of extras. I use flake8 because it's easy, well-established, actively maintained, and there is [_a sweet vim plugin_](https://github.com/janko-m/vim-test) that supports it.

flake8 is very configurable. For a some of my work projects, we have a different flake8 configuration for tests and source code. This lets us do things like have long lines and tabular alignment in tests where it wouldn't be acceptable in the source. However, for packages you are releasing to the community _it's best to just use the vanilla pep8 guidelines_.

Checking a single file or an entire directory is really simple:

**Quick Note**: Don't target your root folder with flake8! It will check all the packages in your virtualenv. Not every package developer cares about community accepted style, so you might have a lot of warnings!

In [17]:
! flake8 hippy_chat

In [18]:
! flake8 tests

tests/test_handler.py:88:80: E501 line too long (90 > 79 characters)


### Ignoring lines
flake8 has a nice feature where you can add a special `# noqa` comment at the end of a line that you don't want to be checked. It's kinda ugly, but that's good because it helps keep you from overusing it.

You can also ignore a whole source file by adding `# flake8: noqa` anywhere in the file

In [19]:
! cat $stuff_dir/test_handler_with_noqa.py

import collections
import json
import logging
import pytest
import requests

from hippy_chat.handler import HipchatHandler, HIPCHAT_DEFAULT_TITLE


class TestHipchatHandler:

    @pytest.fixture(autouse=True)
    def setup(self):
        """
        Sets up the logger for the tests in this class. Is automatically
        invoked before each test function is called
        """
        self.logger = logging.getLogger('TestHipchatLogger')
        self.logger.setLevel(logging.DEBUG)

    @pytest.yield_fixture
    def dummy_handler(self):
        """
        Provides a dummy instance of a HipchatHandler
        """
        dummy_url = 'https://some.fake.server.hipchat.com'
        dummy_room_name = 'some-room'
        dummy_token = 'SOME_TOKEN'

        dummy_handler = HipchatHandler(dummy_url, dummy_room_name, dummy_token)
        self.logger.addHandler(dummy_handler)
        yield dummy_handler
        self.logger.removeHandler(dummy_handler)

    def test___init__(self, dummy_handler):
 

In [20]:
! cp $stuff_dir/test_handler_with_noqa.py tests/test_handler.py
! flake8 tests

## Adding Documentation
Documentation is important for any open source package. For packages of any degree of complexity, documentation **must** extend beyond comments in your code and your README. A lot of times folks avoid writing documentation because getting it all set up is tricky.

The thing is: it isn't

Adding documentation is important for making your package accessible to the community. Some folks won't use your package only because they don't want to read your source code. So, make it easy for them by producing friendly, human-readable docs.

### Why Sphinx?
Most of the Python universe uses [Sphinx](http://www.sphinx-doc.org/en/stable/) for building documentation. In fact, most of the material on python.org is created with Sphinx. Sphinx has a lot of useful plugins, but the one I'm most partial to is called [apidoc](http://www.sphinx-doc.org/en/1.5.1/man/sphinx-apidoc.html). The apidoc plugin can produce documentation pages from your projects module, class, and function docstrings.

This is great because it means your api documenation is produced dynamically, and it encourages you to write good docstrings!

### Getting started with sphinx-quickstart
There's some scaffolding that needs to be put into place first, and Sphinx offers a nice tool that will get you started quickly. It's called sphinx-quickstart. Sphinx-quickstart is an interactive tool that will ask you about what options you want. It produces some boiler-plate doc structure, and, most importantly, it produces a config file for sphinx.

You can also specify all of the options on the command line, which I'm going to do here.

Don't be intimidated by all the options. We're only going override a few defaults, and we're going to blow a lot of stuff away. So, buckle up!

In [21]:
! sphinx-quickstart --help

Sphinx v1.5.2
Usage: sphinx-quickstart [options] [projectdir]

Options:
  --version             show program's version number and exit
  -h, --help            show this help message and exit
  -q, --quiet           quiet mode

  Structure options:
    --sep               if specified, separate source and build dirs
    --dot=DOT           replacement for dot in _templates etc.

  Project basic options:
    -p PROJECT, --project=PROJECT
                        project name
    -a AUTHOR, --author=AUTHOR
                        author names
    -v VERSION          version of project
    -r RELEASE, --release=RELEASE
                        release of project
    -l LANGUAGE, --language=LANGUAGE
                        document language
    --suffix=SUFFIX     source file suffix
    --master=MASTER     master document name
    --epub              use epub

  Extension options:
    --ext-autodoc       enable autodoc extension
    --ext-doctest       enable doctest extension
    --ext-inter

In [22]:
docs_dir = os.path.join(root_dir, 'docs')

# --sep tells quickstart to separate out the docs source and build directories
# I'll explain the CHANGEME values shortly
# -v is for the project version, not to be confused with the sphinx-quickstart --version option...
# I'm going to run this in a separate terminal:'
# $ sphinx-quickstart docs

### Some immediate adjustments
The first thing I'm going to do is to do away with the build directory. In this talk, I'm going to follow my usual pattern which is to build and host my documentation online, so I will never actually build the docs pages locally. I'm also going to then move the contents of the source directory directly into docs, since there won't be any need for another level of folder containment:

In [ ]:
! rm -rf $docs_dir/build
! mv $docs_dir/source/* $docs_dir/
! rm -rf $docs_dir/source
! ls $docs_dir/
! cat $docs_dir/conf.py

### Updating the sphinx configuration
Next, I'm I'm going to change the generated conf.py file so that it will use our dynamic .project_metadata.json

This is a convenient thing to do, because it allows us to keep metadata in one place so that our docs pages and setup.py always agree:

While I'm at it, I'm going to remove the options for LaTeX, man pages, and Texinfo output since we won't be using those:

In [23]:
! cp $stuff_dir/conf.py $docs_dir
! cat $docs_dir/conf.py

cat: /Users/tbeck/work/hippy-chat/docs/conf.py: Not a directory


### Generating api documentation with sphinx-apidoc
Next, we're going to generate the api documentation using the sphinx-apidoc tool

In [24]:
! sphinx-apidoc --help

Usage: sphinx-apidoc [options] -o <output_path> <module_path> [exclude_pattern, ...]

Look recursively in <module_path> for Python modules and packages and create
one reST file with automodule directives per package in the <output_path>.

The <exclude_pattern>s can be file and/or directory patterns that will be
excluded from generation.

Note: By default this script will not overwrite already created files.

Options:
  -h, --help            show this help message and exit
  -o DESTDIR, --output-dir=DESTDIR
                        Directory to place all output
  -d MAXDEPTH, --maxdepth=MAXDEPTH
                        Maximum depth of submodules to show in the TOC
                        (default: 4)
  -f, --force           Overwrite existing files
  -l, --follow-links    Follow symbolic links. Powerful when combined with
                        collective.recipe.omelette.
  -n, --dry-run         Run the script without creating files
  -e, --separate        Put documentation for each mo

In [ ]:
# I like to restrict the TOC depth to 2. For this talk, that won't matter
# --force because, why not?
# for large projects with lots of modules and docs, use separate. For this, no
# generally, you shouldn't use the --private flags...I mean, 'private' methods are private for a reason, right?
# I don't really understand **why** you wouldn't want your module documentation before it's submodules...
! sphinx-apidoc    \
--output-dir=docs  \
--maxdepth=2       \
--force            \
--module-first     \
--suffix=.rst      \
hippy_chat

### Tweaking and viewing the api docs
_I always delete modules.rst. Can't see where it's really every needed or even used_.

The generated module pages are not interesting and don't really need to be modified:

In [ ]:
! rm $docs_dir/modules.rst
! cat $docs_dir/hippy_chat.rst
! cat $docs_dir/hippy_chat.exec.rst

### Adding an overview
I like to write an overview as a separate document. It should include mostly the same data as the README, but embedded in the docs pages instead. It should be simple, and just go over the functionality that your package provides

**Note**: Do not try to `include` or link to README from your doc pages. It doesn't work well (it's disabled on github), and includes are not supported universally


In [ ]:
! rm $docs_dir/modules.rst
! cat $docs_dir/hippy_chat.rst
! cat $docs_dir/hippy_chat.exec.rst

In [25]:
! cp $stuff_dir/overview.rst $docs_dir/
! cat $docs_dir/overview.rst

cp: /Users/tbeck/work/hippy-chat/docs is not a directory
cat: /Users/tbeck/work/hippy-chat/docs/overview.rst: Not a directory


### Adding a quickstart
It's also nice to have a Quickstart document to help your users get going with your package quickly:

In [ ]:
! cp $stuff_dir/quickstart.rst $docs_dir/
! cat $docs_dir/quickstart.rst

### Customizing (rebuilding) the index page
Finally, you will want to basically rebuild your index page. I like to set it up like this:

In [ ]:
! cp $stuff_dir/index.rst docs/
! cat docs/index.rst

### Viewing rendered docs pages
Now that the docs are all built, wanna see what your rendered docs will look? I have an app for that.

I used to use the [restview](https://pypi.python.org/pypi/restview) app to view ReStructuredText files as I edited them. It's an awesome app, but it doesn't use sphinx to render. And I wanted to see sphinx doucments.

So, for fun, I built an app that can do it. There's some fun backstory there, but the long and the short of it is that [sphninx-view](http://sphinx-view.readthedocs.io/en/latest/) works pretty well for this:

In [ ]:
! pip install sphinx-view

In [ ]:
! sphinx-view $docs_dir

## Gitting the code up to github
I'm going to assume that folks listening to this talk are familiar with git and github. If you need to learn about these tools, you can [get started here](https://help.github.com/articles/git-and-github-learning-resources/). I'm going to skip these details in the interst of time.

You should have a github account set up with an ssh key added already.

The first thing you will need to do is [create a new repository](https://github.com/new) on github for your project

### Things todo after creating the repo
Once that is done, we will need to:
* initialize our directory as a git repo
* add the *right* files to the repo
* make our first commit
* add the github repo as a remote
* push our first commit up to github

In [ ]:
! git init
! git add              \
CHANGELOG.rst          \
MANIFEST.in            \
README.rst             \
setup.py               \
.project_metadata.json \
.hipchat.json          \
docs/conf.py           \
docs/*.rst             \
hippy_chat/*.py        \
hippy_chat/exec/*.py   \
tests/*.py
! git commit -m "First commit"
! git remote add origin git@github.com:pip-install-this/hippy-chat.git
! git push -u origin master

### Check out your new github repo
Now we can see the hosted git repo at https://github.com/pip-install-this/hippy-chat

## Documentation Hosting
There are a few different documentation hosting services out there. I have found [readthedocs.org](https://readthedocs.org/) to be my preference. It offers free hosting, natively supports ReStructuredText, has automatic build support, and renders my documents nicely.

This talk assumes that the user already has a readthedocs account set up

### Getting the docs up to readthedocs.org
* Navigate to [readthedocs.org](https://readthedocs.org/)
* Connect your readthedocs account to your github account
* Click the admin button and fill it out.
* Click the down-arrow by your user name in the upper-right and select ['My Projects'](https://readthedocs.org/dashboard/)
* Click the ['import a project'](https://readthedocs.org/dashboard/import/?) button
* If the project doesn't show up, click the 'refresh' button. It should now appear there
* Click the '+' button by your github project's name.
* Select 'Python' in the 'programming languages' box, and make sure 'Sphinx html' is selected in the 'Documentation type' box
* Click 'Finish'
* Once the build is done, click 'View your Documentation' button.

### Viewing the docs pages in their new home
That's it. Now, your documentation is built at http://hippy-chat.readthedocs.io. There are other settings that you can change like which branch the documentation is built from, what version of python interpreter should be used, the default version to show, and others. For now, we will just leave them as they are. Note that the documentation should rebuild _automatically_ any time a new commit is pushed to the branch your documentation is built from

## Continuous Integration
Again, there are lots and lots of different continuous integration tools. I like [travis-ci](https://travis-ci.org/) because it's been around a long time, it's really stable, well known (read, well documented), and offers a lot of free integration with github.

This talk assumes that a user account on travis-ci.org is already set up.

### Getting CI going with travis-ci
* Navigate to [travis-ci.org](https://travis-ci.org)
* Click the '+' button next to 'My Repositories' on the far left
* If your repo doesn't show up, click the 'sync-account' button
* There should be a slider switch next to the repo name. Activate this, then click on the repo name
* You'll see a notice that there are 'No builds for this repository'.
* If you click the 'read the docs' button it will tell you that you nee to setup a .travis.yml file.
* I have one prepared ahead of time
* After it is added, it will need to be committed and pushed to the repo


In [26]:
! cp $stuff_dir/.travis.yml $root_dir/
! cat $root_dir/.travis.yml

language: python
python:
- '3.4'
- '3.5'
install:
- pip install --upgrade pip
- pip install .[dev]
before_script:
- flake8 hippy_chat
- flake8 tests
script: py.test tests


In [ ]:
! git add .travis.yml
! git commit -m "Added travis-ci config"
! git push

### Seeing the results on travis-ci.org
That's it! Now, you should [see that travis-ci has already started a build](https://travis-ci.org/pip-install-this/hippy-chat) for your master branch.

### Checking commits on github with travis-ci
One of the nice ways that github.com and travis-ci.org integrate is by checking pull-requests for a successful build prior to merging. By enabling this feature, you can ensure that any pull-request passes the basic style checking and test suite as a _first step_ to ensuring that the code is right
* Navigate to the [github page](https://github.com/pip-install-this/hippy-chat)
* Click the [settings tab](https://github.com/pip-install-this/hippy-chat/settings)
* Go to [Branches](https://github.com/pip-install-this/hippy-chat/settings/branches)
* In the 'Protected Branches' section, click 'choose a branch' and select ['master'](https://github.com/pip-install-this/hippy-chat/settings/branches/master)
* Enable 'protect this branch'
* Enable 'Require pull request reviews before merging' and 'Require status checks to pass before merging'
* Don't include administrators
* Click on the 'continuous-integration/travis-ci' to enable the travis check before merging




### Making a commit on a branch to try out commit checks
Before we can see travis-ci and github in action, we'll need a new commit on a new branch:

In [ ]:
! git checkout -b pip-install-this/changelog-updates
! echo " - Added project documentation pages" >> $root_dir/CHANGELOG.rst
! echo " - Set up travis-ci integration" >> $root_dir/CHANGELOG.rst
! git add $root_dir/CHANGELOG.rst
! git commit -m "Added a couple of entries to the CHANGELOG"
! git push --set-upstream origin pip-install-this/changelog-updates

### Observing commit checks in the wild
Now, [create a new pull request for the latest commit](https://github.com/pip-install-this/hippy-chat/compare)
* select the branch to 'compare'
* click 'create pull-request'
* click 'create pull-request' again...

If you wait a while, the checks will either pass or fail. If the checks pass, the branch can be merged.

You can watch the status of your checks at [travis-ci.org](https://travis-ci.org/pip-install-this/hippy-chat/builds)

## Publishing to PyPI
Once you have a reasonably stable package with good documentation and testing, you are _probably_ ready to publish your package on [PyPI](https://pypi.python.org/pypi)

Again, this talk is assuming that a user account has already been created on PyPI

It is good wisdom to publish your package to the [PyPI Testing Site](https://testpypi.python.org/pypi) before you actually submit it to the main registry. We're not going to do that in this talk for the sake of time, but you should in real life. This will allow you to resolve problems without worrying about anyone having already installed your package

### Setting up a .pypirc file
This file contains your credentials for PyPI. It is used when you need to publish 'the old fashioned way'. It will be useful in the initial publication to PyPI

In [27]:
! cp $stuff_dir/.pypirc ~/
! cat ~/.pypirc
! echo "password: $(cat ~/.pip-install-this.pw)" >> ~/.pypirc

[distutils]
index-servers =
    pypi

[pypi]
repository: https://www.python.org/pypi
username: pip-install-this


### Your first build
We're going to build a [wheel](http://pythonwheels.com/) and source distribution of our package for the first time. Don't worry, we won't be doing this maybe ever again, because we're going to let automated tools help us. However, for this first time, we have to do it by hand

There is no option (_that I know of_) to build using pip, so we are going to use the ol' setuptools approach:

In [ ]:
! python setup.py sdist bdist_wheel

### Uploading builds to PyPI
Next, we will need to upload our package to PyPI. Again, we can use distutils/setuptools to do this:

In [ ]:
! python setup.py sdist bdist_wheel upload

### Taking a peek at your published package on PyPI
If you navigate back to [PyPI](https://pypi.python.org/pypi) now, you should be able to see a link to your package on the right. You can even try to pip install your package as well

## Automatic publishing of new versions
The travis-ci service offers an option to automatically publish new tagged versions of your repo to PyPI. This is incredibly handy because it means you can update your package's version just by pushing a new tag.  You can [read more about this on the travis-ci doc pages](https://docs.travis-ci.com/user/deployment/pypi/)



### Setting up travis for automatically publishing
You will need to configure your `.travis.yml` file to enable automatic publishing. You can do this by hand, but there is a command-line tool that can do it for you.

It's a ruby app that's installable as a gem (`gem install travis`). I already have that set up, so you'll just have to take my word for it.

This is an interactive command-line tool, and I've had no success running it in non-interactive mode. So, we go to a terminal here and type:

`$ travis setup pypi`

### Setting up travis for automatically publishing continued
Once we have updated the .travis.yml file with deployment settings, all we need to do for travis-ci to use the new configuration is to push the commit to github

In [ ]:
! git checkout master
! git add .travis.yml
! git commit -m "Updated the .travis.yml file with deployment settings"
! git push

## Deploying a new version automagically
In order to deploy a new version, we will need to create a new tag in our git repo. This tag should include an updated version from our `.project_metadata.json` file, and the tag needs to be pushed out to our github repo

In [ ]:
! cp $stuff_dir/.project_metadata.json.bumped $root_dir/.project_metadata.json
! cat $root_dir/.project_metadata.json
! git add $root_dir/.project_metadata.json
! git tag -a v0.2.0 -m "hippy-chat version 0.2.0"
! git push --tags

## Observing the new deploy
We should be able to see the tag now on [github.com](https://github.com/pip-install-this/hippy-chat/) under 'tags'

We can follow the build of our new tagged version on [travis-ci.org](https://travis-ci.org/pip-install-this/hippy-chat)

Once it is finished, we can see the results on [PyPI](https://pypi.python.org/pypi?%3Aaction=pkg_edit&name=hippy-chat)

## Wrapping Up
Well, that's about it for this talk. There's a lot here, and hopefully, we're reviewing this slide around the 60 minute mark, and really hopefully we actually made it here.

### Stuff I'd love to talk about, but ran out of time
* pip vs easy_install
* conda. Supposed to be really good...haven't tried it yet
* Where's my requirements.txt file?

### Discussion
* Questions?
* Where did I screw up?
* What alternative tools do you like and why?
* What else should be included here?

## Thank you!